In [1]:
import joblib
from pathlib import Path
import argparse
import os
import datetime
from dataset.bbox_dataset import get_datasets, get_kfold_dataset
from dataset.create_kfolds import create_kfolds
from models.bbox_dispatcher import create_model
from metrices.iou import IoU, IoUMetric
from train_bbox import __get_callbacks__

import tensorflow as tf

import tensorflow as tf

import os
import datetime

from PIL import Image
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [2]:
dataset_path = '../fyp/ccpd_sample/content/home/booy/booy/ccpd_dataset/ccpd_base/'
folds = 3

In [3]:
fold_data = create_kfolds(Path(dataset_path), folds)
fold = 0

dataset directory:../fyp/ccpd_sample/content/home/booy/booy/ccpd_dataset/ccpd_base
number of files: 9999 fold size : 3333


In [4]:
model_name = 'v16'
input_shape = target_size = (480,480)
BATCH_SIZE = 32

In [5]:
model = create_model(model_name, input_shape + (3, ))
train_dataset, test_dataset = get_kfold_dataset(fold_data, fold, input_shape, BATCH_SIZE)


In [6]:
model.save('tmp')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tmp/assets


In [7]:
model_1 = tf.keras.models.load_model('tmp',custom_objects={'IoUMetric':IoUMetric})

In [ ]:
def plot_image(dataset,idx=0):
  images, Y = next(iter(dataset))
  image = images[1]
  y = Y[1]
  image = (image.numpy()*255).astype(np.uint8)
  y = (y.numpy()*target_size[0]).astype(int)

  plt.imshow(image, interpolation='nearest')
  # Get the current reference
  ax = plt.gca()
  x_center, y_center, w, h = y
  x1 = x_center - (w/2)
  y1 = y_center - (h/2)

  # Create a Rectangle patch
  rect = Rectangle((x1,y1),w,h,linewidth=1,edgecolor='r',facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)
  plt.show()


In [ ]:
plot_image(test_dataset)

In [ ]:
callbacks = __get_callbacks__(model_name)
model.fit(train_dataset,
      validation_data=test_dataset,
      callbacks=[callbacks],
      epochs=60)


In [ ]:
model_0 = tf.keras.models.load_model('cv_s1_v28/saved_models/simple_bbox/v28_0')
model_1 = tf.keras.models.load_model('cv_s1_v28/saved_models/simple_bbox/v28_1')
model_2 = tf.keras.models.load_model('cv_s1_v28/saved_models/simple_bbox/v28_2')

In [ ]:
IMG, Y = next(iter(test_dataset))

In [ ]:
def show_prediction(IMG,BBOX,idx=0):
    image = IMG[idx]
    y = BBOX[idx]
    image = (image.numpy()*255).astype(np.uint8)
    y = (y.numpy()*target_size[0]).astype(int)

    plt.imshow(image, interpolation='nearest')
    # Get the current reference
    ax = plt.gca()
    x_center, y_center, w, h = y
    x1 = x_center - (w/2)
    y1 = y_center - (h/2)

    # Create a Rectangle patch
    rect = Rectangle((x1,y1),w,h,linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)
    plt.show()


In [ ]:
show_prediction(IMG,Y)

In [ ]:
Y_hat = model_0(IMG)
show_prediction(IMG,Y_hat)

In [ ]:
Y_hat = model_1(IMG)
show_prediction(IMG,Y_hat)

In [ ]:
Y_hat = model_2(IMG)
show_prediction(IMG,Y_hat)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

@tf.function
def to_box_cords(bboxs):
    X = bboxs[:,0]
    Y = bboxs[:,1]
    W = bboxs[:,2]
    H = bboxs[:,3]
    X_1 = X - (W/2)
    Y_1 = Y - (H/2)
    X_2 = X + (W/2)
    Y_2 = Y + (H/2)
    return tf.stack([Y_1,X_1,Y_2,X_2],axis=1)

In [ ]:
box_true = to_box_cords(Y)
box_pred = to_box_cords(Y_hat)

In [ ]:
boxA = box_true * target_size[0]
boxB = box_pred * target_size[0]

In [ ]:
boxes = tf.stack([boxA,boxB],axis=1)

In [ ]:
def __iou__(boxes):
    boxA = boxes[0,:]
    boxB = boxes[1,:]
    xA = tf.math.maximum(boxA[0], boxB[0])
    yA = tf.math.maximum(boxA[1], boxB[1])
    xB = tf.math.minimum(boxA[2], boxB[2])
    yB = tf.math.minimum(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = tf.math.maximum(0, xB - xA + 1) * tf.math.maximum(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
ious = tf.map_fn(fn=__iou__, elems=boxes)
iou = tf.reduce_sum(ious) / Y.shape[0]

In [ ]:
iou

In [ ]:
class IoU(tf.keras.metrics.Metric):
    def __init__(self,target_size, threshold=0.7,name='iou',**kwargs):
        super(IoU, self).__init__(name=name, **kwargs)
        self.target_size = target_size
        self.threshold = threshold
        self.iou = self.add_weight(name='iou', initializer='zeros')
    
    @tf.function
    def __iou__(self,boxes):
        boxA = boxes[0,:]
        boxB = boxes[1,:]
        xA = tf.math.maximum(boxA[0], boxB[0])
        yA = tf.math.maximum(boxA[1], boxB[1])
        xB = tf.math.minimum(boxA[2], boxB[2])
        yB = tf.math.minimum(boxA[3], boxB[3])
        # compute the area of intersection rectangle
        interArea = tf.math.maximum(0.0, xB - xA + 1) * tf.math.maximum(0.0, yB - yA + 1)
        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
        # return the intersection over union value
        return iou
        
    @tf.function
    def to_box_cords(self,bboxs):
        X = bboxs[:,0]
        Y = bboxs[:,1]
        W = bboxs[:,2]
        H = bboxs[:,3]
        X_1 = X - (W/2)
        Y_1 = Y - (H/2)
        X_2 = X + (W/2)
        Y_2 = Y + (H/2)
        return tf.stack([Y_1,X_1,Y_2,X_2],axis=1)
    
    def update_state(self, Y, Y_hat, sample_weight=None):
        box_true = self.to_box_cords(Y)
        box_pred = self.to_box_cords(Y_hat)
        
        boxA = box_true * self.target_size[0]
        boxB = box_pred * self.target_size[0]
        boxes = tf.stack([boxA,boxB],axis=1)
        
        ious = tf.map_fn(fn=self.__iou__, elems=boxes)
        iou = tf.reduce_sum(ious) / Y.shape[0]
        
        self.iou.assign_add(iou)
        
    def result(self):
        return self.iou

In [ ]:
m = IoU((480,480))
m.update_state(Y,Y_hat)

In [ ]:
m.result().numpy()